![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [51]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Load dataset and view
rental_df = pd.read_csv('rental_info.csv')
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [52]:
# Convert 'rental_date' and 'return_date' to datetime objects
rental_df['rental_date'] = pd.to_datetime(rental_df['rental_date'])
rental_df['return_date'] = pd.to_datetime(rental_df['return_date'])

# Calculate rental length in days
rental_df['rental_length_days'] = (rental_df['return_date'] - rental_df['rental_date']).dt.days

# Check dtype changes
rental_df[['rental_date', 'return_date']].dtypes

rental_date    datetime64[ns, UTC]
return_date    datetime64[ns, UTC]
dtype: object

In [53]:
# Create dummy variables for 'special_features'
rental_df['deleted_scenes'] = np.where(rental_df["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_df['behind_the_scenes'] = np.where(rental_df["special_features"].str.contains("Behind the Scenes"), 1, 0)

In [54]:
# Create list for columns to drop
cols_drop = ["special_features", "rental_length_days", "rental_date", "return_date"]

# Split into feature and target sets
X = rental_df.drop(cols_drop, axis=1)
y = rental_df["rental_length_days"]

# Further split into training and test data
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size= 0.2, random_state= 9)

In [55]:
# Instantiate LinearRegression model 
ols_model = LinearRegression()

# Train the model
ols_model.fit(X_train, y_train)

# Predict on the test set
y_pred_ols = ols_model.predict(X_test)
mse_ols = mean_squared_error(y_test, y_pred_ols)
mse_ols

2.9417238646975883

In [56]:
# Instantiate RandomForest ensemble
rf_model = RandomForestRegressor(n_estimators= 100, random_state= 9)
# Fit the RF model
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_rf

2.030141907417274

In [57]:
# Display the model with the lowest MSE:
best_model = rf_model
best_mse = mse_rf